In [72]:
from this import d
import numpy as np
import pydicom as dicom
from PIL import Image
import glob
import matplotlib.pyplot as plt
import cv2
import h5py
import os
from tqdm import tqdm
## the directory with datatset
root_dir = "/home/jaye/Documents/MedicalDatasets/CHAOS/Train_Sets/"
files = glob.glob(root_dir)
image_path = []
label_path = []
CT_test_list = ['003','004','007','009','011','012','013','015','017','020']
test_list = ['004','006','007','009','011','012','014','016','017','018','023',
             '024','025','026','027','028','029','030']

data_model = "MR"
# for CT
if data_model == "CT":
    CT_root = os.path.join(root_dir,"CT")
    for each in os.listdir(CT_root):
        each_path = os.path.join(CT_root,each) #"/home/jaye/Documents/MedicalDatasets/CHAOS/Train_Sets/CT/1/"
        image_lst = os.listdir(os.path.join(each_path,'DICOM_anon')) # DICOM_anno and Ground
        label_lst = os.listdir(os.path.join(each_path,'Ground'))
        for i in range(len(image_lst)):
            image_path.append(os.path.join(each_path,'DICOM_anon',image_lst[i]))
            label_path.append(os.path.join(each_path,'Ground',image_lst[i]))
#for MRI, only T2SPIR is loaded
elif data_model == "MR":
    MR_root = os.path.join(root_dir,"MR")
    for each in os.listdir(MR_root):
        each_path = os.path.join(MR_root,each)
        if "T2SPIR" in os.listdir(each_path):
            image_lst = os.listdir(os.path.join(each_path,"T2SPIR/DICOM_anon"))
            for i in range(len(image_lst)):
                image_path.append(os.path.join(each_path,'T2SPIR/DICOM_anon',image_lst[i]))
                label_path.append(os.path.join(each_path,'T2SPIR/Ground',image_lst[i].replace('.dcm','.png')))
            
print(image_path[0])
print(label_path[0])
image = dicom.read_file(image_path[3]).pixel_array
label = np.array(Image.open(label_path[3]).convert('L'))
print(image.shape)
print(label.shape)
os.makedirs('/home/jaye/Documents/MedicalDatasets/CHAOS/processed_training/train_npz',exist_ok=True)
os.makedirs('/home/jaye/Documents/MedicalDatasets/CHAOS/processed_training/test_npz',exist_ok=True)
os.makedirs('/home/jaye/Documents/MedicalDatasets/CHAOS/processed_training/lists_ProstateX2/',exist_ok=True)
train_file = open('/home/jaye/Documents/MedicalDatasets/CHAOS/processed_training/lists_ProstateX2/train.txt','w')
test_file = open('/home/jaye/Documents/MedicalDatasets/CHAOS/processed_training/lists_ProstateX2/test.txt','w')
for i in tqdm(range(len(image_lst))):
    name_case = image_path[i].split("/")[-1].replace('.dcm','')
    image = dicom.read_file(image_path[i]).pixel_array
    label = np.array(Image.open(label_path[i]).convert('L'))
    image = image/255
    # train and val dataset split
    order = np.arange(len(image_lst))
    np.random.shuffle(order)
    test_order = order[:int(len(image_lst)/4)]
    # train dataset 
    if (name_case) not in (np.array(image_lst).take(test_order)):
        case_image = cv2.resize(image,(224,224),interpolation=cv2.INTER_NEAREST)
        case_label = cv2.resize(label,(224,224),interpolation=cv2.INTER_NEAREST)
        np.savez("/home/jaye/Documents/MedicalDatasets/ProstateX2/processed_training/train_npz/" + str(name_case),image = case_image, label=case_label)
        train_file.write(str(name_case) +'.npz'+'\n')

    else:
        name_case = name_case.split('-')[:-1]
        case_image = cv2.resize(image,(224,224),interpolation=cv2.INTER_NEAREST)
        case_label = cv2.resize(label,(224,224),interpolation=cv2.INTER_NEAREST)
        np.savez("/home/jaye/Documents/MedicalDatasets/ProstateX2/processed_training/test_npz/" + str(name_case), image = case_image, label=case_label)
        test_file.write(str(name_case) +'.npz'+'\n')
train_file.close()
test_file.close()

/home/jaye/Documents/MedicalDatasets/CHAOS/Train_Sets/MR/21/T2SPIR/DICOM_anon/IMG-0001-00004.dcm
/home/jaye/Documents/MedicalDatasets/CHAOS/Train_Sets/MR/21/T2SPIR/Ground/IMG-0001-00004.png
(256, 256)
(256, 256)


  0%|          | 0/39 [00:00<?, ?it/s]

In [47]:
x = [0,1,2]
y = [1,2]
print(np.array(x).take(y))


[1 2]
